# 🚀 Swin-Base + MobileViT (Kvasir) with Optuna TPE

## Architecture
- **Swin-Base**: 1024-dim features (ImageNet pretrained)
- **MobileViT**: 384-dim features (**Kvasir pretrained** - custom architecture)
- **Fusion**: 1024 + 384 = 1408-dim -> 256-dim
- **Classifier**: CatBoost with Optuna TPE optimization

## Expected Results
- **Accuracy**: ~88-90% (with Kvasir MobileViT)
- **AUROC**: ~98-99%
- **Training Time**: ~45-50 minutes

In [ ]:
!pip install -q optuna catboost scikit-learn
print("✅ Packages installed")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
weights_path = '/content/drive/MyDrive/TripleHybridModel/mobilevit_kvasir_v2_best_optuna.pth'

if os.path.exists(weights_path):
    print(f"✅ Found MobileViT Kvasir weights")
else:
    print(f"❌ Weights not found")
    weights_path = None

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np
from pathlib import Path
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, log_loss
from sklearn.preprocessing import label_binarize
import optuna
from catboost import CatBoostClassifier
import time, json

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🎮 Device: {device}")

In [ ]:
if not os.path.exists('kvasir-dataset-v2'):
    !wget --no-check-certificate https://datasets.simula.no/downloads/kvasir/kvasir-dataset-v2.zip
    !unzip -q kvasir-dataset-v2.zip
    print("✅ Dataset ready!")

## 🏗️ Custom MobileViT (same as Swin-Small notebook)

In [ ]:
# [Copy the exact same MobileViT code from Swin-Small notebook]
# Helper functions
def conv_1x1_bn(inp, oup):
    return nn.Sequential(nn.Conv2d(inp, oup, 1, 1, 0, bias=False), nn.BatchNorm2d(oup), nn.Hardswish())

def conv_nxn_bn(inp, oup, kernal_size=3, stride=1):
    return nn.Sequential(nn.Conv2d(inp, oup, kernal_size, stride, 1, bias=False), nn.BatchNorm2d(oup), nn.Hardswish())

class PreNorm(nn.Module):
    def __init__(self, dim, fn):
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.fn = fn
    def forward(self, x, *args, **kwargs):
        return self.fn(self.norm(x), *args, **kwargs)

class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim, dropout=0.):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(dim, hidden_dim), nn.Hardswish(), nn.Dropout(dropout), nn.Linear(hidden_dim, dim), nn.Dropout(dropout))
    def forward(self, x):
        return self.net(x)

class Attention(nn.Module):
    def __init__(self, dim, heads=8, dim_head=64, dropout=0.):
        super().__init__()
        self.inner_dim = dim_head * heads
        self.heads, self.scale = heads, dim_head ** -0.5
        self.attend = nn.Softmax(dim=-1)
        self.dropout = nn.Dropout(dropout)
        self.to_qkv = nn.Linear(dim, self.inner_dim * 3, bias=False)
        self.to_out = nn.Sequential(nn.Linear(self.inner_dim, dim), nn.Dropout(dropout)) if not (heads == 1 and dim_head == dim) else nn.Identity()
    def forward(self, x):
        qkv = self.to_qkv(x).chunk(3, dim=-1)
        q, k, v = map(lambda t: t.reshape(t.shape[0], t.shape[1], self.heads, t.shape[2] // self.heads).transpose(1, 2), qkv)
        dots = torch.matmul(q, k.transpose(-1, -2)) * self.scale
        attn = self.dropout(self.attend(dots))
        out = torch.matmul(attn, v).transpose(1, 2).reshape(out.shape[0], out.shape[1], self.inner_dim)
        return self.to_out(out)

class TransformerBlock(nn.Module):
    def __init__(self, dim, heads, dim_head, mlp_dim, dropout=0.):
        super().__init__()
        self.attn = PreNorm(dim, Attention(dim, heads=heads, dim_head=dim_head, dropout=dropout))
        self.ff = PreNorm(dim, FeedForward(dim, mlp_dim, dropout=dropout))
    def forward(self, x):
        return x + self.ff(x + self.attn(x))

class MobileNetV2Block(nn.Module):
    def __init__(self, inp, oup, stride, expansion):
        super().__init__()
        self.stride, hidden_dim = stride, int(round(inp * expansion))
        self.use_res_connect = self.stride == 1 and inp == oup
        layers = [nn.Conv2d(inp, hidden_dim, 1, 1, 0, bias=False), nn.BatchNorm2d(hidden_dim), nn.Hardswish()] if expansion != 1 else []
        layers.extend([nn.Conv2d(hidden_dim, hidden_dim, 3, stride, 1, groups=hidden_dim, bias=False), nn.BatchNorm2d(hidden_dim), nn.Hardswish(), nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False), nn.BatchNorm2d(oup)])
        self.conv = nn.Sequential(*layers)
    def forward(self, x):
        return x + self.conv(x) if self.use_res_connect else self.conv(x)

class MobileViTBlock(nn.Module):
    def __init__(self, dim, depth, channel, kernal_size, patch_size, mlp_dim, dropout=0.):
        super().__init__()
        self.ph, self.pw = patch_size
        self.conv1 = conv_nxn_bn(channel, channel, kernal_size)
        self.conv2 = conv_1x1_bn(channel, dim)
        self.transformer = nn.Sequential(*[TransformerBlock(dim, 4, 8, mlp_dim, dropout) for _ in range(depth)])
        self.conv3 = conv_1x1_bn(dim, channel)
        self.conv4 = conv_nxn_bn(2 * channel, channel, kernal_size)
    def forward(self, x):
        y = x.clone()
        x = self.conv2(self.conv1(x))
        B, D, H, W = x.shape
        pad_h, pad_w = (self.ph - (H % self.ph)) % self.ph, (self.pw - (W % self.pw)) % self.pw
        if pad_h > 0 or pad_w > 0:
            x = F.pad(x, (0, pad_w, 0, pad_h))
            H, W = H + pad_h, W + pad_w
        h_split, w_split = H // self.ph, W // self.pw
        x = x.reshape(B, D, h_split, self.ph, w_split, self.pw).permute(0, 2, 4, 3, 5, 1).contiguous().view(B * h_split * w_split, self.ph * self.pw, D)
        x = self.transformer(x).view(B, h_split, w_split, self.ph, self.pw, D).permute(0, 5, 1, 3, 2, 4).contiguous().view(B, D, H, W)
        if pad_h > 0 or pad_w > 0:
            x = x[:, :, :H-pad_h, :W-pad_w]
        return self.conv4(torch.cat((self.conv3(x), y), 1))

class MobileViT(nn.Module):
    def __init__(self, image_size, num_classes, dims, channels, depths, patch_size=2, mlp_dim_ratio=2):
        super().__init__()
        self.conv1 = conv_nxn_bn(3, channels[0], kernal_size=3, stride=2)
        self.stem = nn.ModuleList([MobileNetV2Block(channels[i], channels[i+1], stride=2 if i in [1,3] else 1, expansion=4) for i in range(4)])
        self.mobilevit_blocks = nn.ModuleList()
        in_channel = channels[4]
        for i in range(len(dims)):
            self.mobilevit_blocks.append(MobileViTBlock(dims[i], depths[i], in_channel, 3, (patch_size, patch_size), dims[i] * mlp_dim_ratio))
            if i < len(dims) - 1:
                self.mobilevit_blocks.append(MobileNetV2Block(in_channel, channels[i+5], stride=2, expansion=4))
                in_channel = channels[i+5]
        self.conv2 = conv_1x1_bn(in_channel, dims[-1])
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(dims[-1], num_classes)
    def forward(self, x):
        x = self.conv1(x)
        for block in self.stem:
            x = block(x)
        for block in self.mobilevit_blocks:
            x = block(x)
        return self.fc(self.pool(self.conv2(x)).flatten(1))

print("✅ MobileViT defined")

In [ ]:
class SwinBaseMobileViTHybrid(nn.Module):
    def __init__(self, mobilevit_weights_path=None):
        super().__init__()
        print("Loading Swin-Base...")
        self.swin = models.swin_b(weights=models.Swin_B_Weights.IMAGENET1K_V1)
        self.swin.head = nn.Identity()
        self.swin_dim = 1024
        
        print("Loading Custom MobileViT...")
        self.mobilevit = MobileViT(image_size=224, num_classes=8, dims=[96, 192, 384], channels=[16, 32, 64, 128, 160, 192, 256, 320, 384, 512], depths=[2, 2, 2])
        if mobilevit_weights_path and os.path.exists(mobilevit_weights_path):
            self.mobilevit.load_state_dict(torch.load(mobilevit_weights_path, map_location='cpu'))
            print("✅ MobileViT Kvasir weights loaded!")
        self.mobilevit.fc = nn.Identity()
        self.mobilevit_dim = 384
        
        for param in self.swin.parameters():
            param.requires_grad = False
        for param in self.mobilevit.parameters():
            param.requires_grad = False
        
        combined_dim = self.swin_dim + self.mobilevit_dim
        self.fusion = nn.Sequential(nn.Linear(combined_dim, 512), nn.ReLU(), nn.Dropout(0.3), nn.Linear(512, 256))
        print(f"✅ Hybrid: {self.swin_dim} + {self.mobilevit_dim} = {combined_dim} -> 256")
    
    def forward(self, x):
        with torch.no_grad():
            return self.fusion(torch.cat([self.swin(x), self.mobilevit(x)], dim=1))

print("✅ Hybrid model defined")

In [ ]:
class KvasirDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = Path(root_dir)
        self.transform = transform
        self.samples = []
        classes = sorted([d.name for d in self.root_dir.iterdir() if d.is_dir()])
        self.class_to_idx = {cls: idx for idx, cls in enumerate(classes)}
        for cls in classes:
            for img in (self.root_dir / cls).glob('*.jpg'):
                self.samples.append((img, self.class_to_idx[cls]))
    def __len__(self): return len(self.samples)
    def __getitem__(self, idx):
        img, label = self.samples[idx]
        return self.transform(Image.open(img).convert('RGB')), label

transform = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor(), transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
dataset_path = 'kvasir-dataset-v2/kvasir-dataset-v2' if os.path.exists('kvasir-dataset-v2/kvasir-dataset-v2') else 'kvasir-dataset-v2'
dataset = KvasirDataset(dataset_path, transform=transform)
idx = list(range(len(dataset)))
labels = [s[1] for s in dataset.samples]
tr_idx, temp = train_test_split(idx, test_size=0.3, stratify=labels, random_state=42)
val_idx, te_idx = train_test_split(temp, test_size=0.5, stratify=[labels[i] for i in temp], random_state=42)
print(f"📊 Dataset: Train={len(tr_idx)}, Val={len(val_idx)}, Test={len(te_idx)}")

In [ ]:
model = SwinBaseMobileViTHybrid(weights_path).to(device).eval()

def extract_features(indices, set_name):
    loader = DataLoader(torch.utils.data.Subset(dataset, indices), batch_size=32, num_workers=2)
    feats, lbls = [], []
    start_time = time.time()
    for img, lbl in tqdm(loader, desc=f"Extracting {set_name}"):
        feats.append(model(img.to(device)).cpu().detach().numpy())
        lbls.append(lbl.numpy())
    return np.vstack(feats), np.concatenate(lbls), time.time() - start_time

print("🔬 Extracting features...")
X_train, y_train, train_time = extract_features(tr_idx, "Train")
X_val, y_val, val_time = extract_features(val_idx, "Val")
X_test, y_test, test_time = extract_features(te_idx, "Test")
print(f"\n✅ Features: {X_train.shape}")
print(f"⏱️ Time: Train={train_time:.1f}s, Val={val_time:.1f}s, Test={test_time:.1f}s")

In [ ]:
optuna.logging.set_verbosity(optuna.logging.WARNING)

def objective(trial):
    params = {'depth': trial.suggest_int('depth', 4, 10), 'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True), 'iterations': trial.suggest_int('iterations', 100, 1000), 'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10), 'border_count': trial.suggest_int('border_count', 32, 255), 'task_type': 'GPU', 'verbose': False, 'random_seed': 42}
    clf = CatBoostClassifier(**params)
    clf.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=50, verbose=False)
    return accuracy_score(y_val, clf.predict(X_val))

print("🔍 Optuna TPE (50 trials)...")
opt_start = time.time()
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=50, show_progress_bar=True)
opt_time = time.time() - opt_start
print(f"\n🏆 Best: {study.best_value*100:.2f}%")

In [ ]:
final_params = study.best_params
final_params.update({'task_type': 'GPU', 'random_seed': 42, 'verbose': True})
final_model = CatBoostClassifier(**final_params)
train_start = time.time()
final_model.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=50)
train_time_final = time.time() - train_start

In [ ]:
def calc_metrics(model, X, y):
    start = time.time()
    y_pred, y_prob = model.predict(X), model.predict_proba(X)
    inf_time = time.time() - start
    y_bin = label_binarize(y, classes=range(8))
    cm = confusion_matrix(y, y_pred)
    tpr_list, fpr_list = [], []
    for i in range(8):
        tp, fn, fp = cm[i, i], cm[i, :].sum() - cm[i, i], cm[:, i].sum() - cm[i, i]
        tn = cm.sum() - tp - fn - fp
        tpr_list.append(tp / (tp + fn) if (tp + fn) > 0 else 0)
        fpr_list.append(fp / (fp + tn) if (fp + tn) > 0 else 0)
    return {'accuracy': accuracy_score(y, y_pred), 'precision': precision_score(y, y_pred, average='weighted', zero_division=0), 'recall': recall_score(y, y_pred, average='weighted', zero_division=0), 'f1_score': f1_score(y, y_pred, average='weighted', zero_division=0), 'auroc': roc_auc_score(y_bin, y_prob, average='weighted', multi_class='ovr'), 'tpr': np.mean(tpr_list), 'fpr': np.mean(fpr_list), 'loss': log_loss(y, y_prob), 'time': inf_time}

train_m = calc_metrics(final_model, X_train, y_train)
val_m = calc_metrics(final_model, X_val, y_val)
test_m = calc_metrics(final_model, X_test, y_test)
print(f"\nTest: {test_m['accuracy']*100:.2f}% | AUROC: {test_m['auroc']*100:.2f}%")

In [ ]:
final_model.save_model('swin_base_kvasir_optuna.cbm')
np.savez('swin_base_kvasir_features.npz', X_train=X_train, y_train=y_train, X_val=X_val, y_val=y_val, X_test=X_test, y_test=y_test)
results = {'architecture': 'Swin-Base + MobileViT (Kvasir)', 'optimization': 'Optuna TPE', 'n_trials': 50, 'best_params': study.best_params, 'best_val_accuracy': study.best_value, 'training_metrics': train_m, 'validation_metrics': val_m, 'test_metrics': test_m, 'timing': {'feature_extraction': {'train': train_time, 'val': val_time, 'test': test_time}, 'optimization': opt_time, 'final_training': train_time_final}}
with open('swin_base_kvasir_optuna_results.json', 'w') as f:
    json.dump(results, f, indent=2, default=float)

from google.colab import files
files.download('swin_base_kvasir_optuna.cbm')
files.download('swin_base_kvasir_features.npz')
files.download('swin_base_kvasir_optuna_results.json')
print("✅ Done!")